In [1]:
!pip install pymongo apscheduler


In [2]:
from pymongo import MongoClient
from datetime import datetime
import time

CONNECTION_STRING = " "
client = MongoClient(CONNECTION_STRING)




In [3]:
log_db = client["log_database"]
movie_db = client["movie_database"]
user_db = client["user_database"]

recommendation_collection = log_db["recommendation_log"]
watch_time_collection = movie_db["user_watch_data"]


In [4]:
print(log_db.list_collection_names())

['recommendation_log']


In [5]:
def extract_watch_duration(minute_mpg):
    """
    Extracts numeric watch duration in seconds from 'minute_mpg' field.
    """
    try:
        minutes = int(minute_mpg.split(".")[0])  # Extract minutes (before ".mpg")
        return minutes * 60  # Convert to seconds
    except:
        return 0  # Default to 0 if parsing fails


In [6]:

def compute_avg_watch_time():
    """
    Computes average watch time for movies watched AFTER the recommendation.
    
    """
    total_watch_time = 0
    total_users = 0

    # Process only a limited number of recommendations (e.g., 1000)
    recommendation_logs = recommendation_collection.find().limit(1000)

    for rec in recommendation_logs:
        user_id = rec.get("user_id")
        recommended_movies = rec.get("recommendation_results", [])

        # Convert timestamp correctly
        rec_timestamp = rec.get("time")
        if isinstance(rec_timestamp, str):  # Convert if it's a string
            try:
                rec_timestamp = datetime.fromisoformat(rec_timestamp.replace("Z", "+00:00"))
            except ValueError:
                continue  # Skip records with invalid timestamps

        # Query watch logs for this user AFTER recommendation
        watch_logs = list(watch_time_collection.find(
            {
                "user_id": user_id,
                "movie_id": {"$in": recommended_movies},
                "time": {"$gte": rec_timestamp}  # Watch time after recommendation
            },
            {"minute_mpg": 1}  # Fetch only required field
        ))

        for watch in watch_logs:
            watch_duration = extract_watch_duration(watch["minute_mpg"])
            total_watch_time += watch_duration
            total_users += 1

    # Compute final average
    avg_watch_time = total_watch_time / total_users if total_users > 0 else 0
    print(f" Average Watch Time AFTER Recommendation: {avg_watch_time:.2f} seconds")
    return avg_watch_time

# Run the function
compute_avg_watch_time()


 Average Watch Time AFTER Recommendation: 991.55 seconds


991.5526472337895

In [7]:
def compute_watch_time_conversion_rate():
    """
    Computes the percentage of users who watched a recommended movie AFTER receiving the recommendation.
    """
    total_recommendations = recommendation_collection.count_documents({})
    users_who_watched = set()

    for rec in recommendation_collection.find().limit(1000):  # Add limit to avoid infinite loops
        user_id = rec.get("user_id")
        recommended_movies = rec.get("recommendation_results", [])
        rec_timestamp = rec.get("time")

        # Handle time format conversion
        if isinstance(rec_timestamp, str):
            try:
                rec_timestamp = datetime.fromisoformat(rec_timestamp.replace("Z", "+00:00"))
            except Exception as e:
                print(f"Skipping user {user_id} due to invalid time format: {e}")
                continue

        # Query for watch logs after the recommendation
        watch_logs_cursor = watch_time_collection.find({
            "user_id": user_id,
            "movie_id": {"$in": recommended_movies},
            "time": {"$gte": rec_timestamp}
        })

        # Convert to list to count results
        watch_logs = list(watch_logs_cursor)

        if len(watch_logs) > 0:
            users_who_watched.add(user_id)

    conversion_rate = len(users_who_watched) / total_recommendations if total_recommendations > 0 else 0
    print(f"Watch Time Conversion Rate AFTER Recommendation: {conversion_rate:.2%}")
    return conversion_rate

# Run it
compute_watch_time_conversion_rate()



Watch Time Conversion Rate AFTER Recommendation: 3.14%


0.0313994593470576

In [8]:
telemetry_collection = log_db["online_evaluation_telemetry"]

In [11]:

def log_online_telemetry():
    """
    Computes evaluation metrics and stores them as telemetry data in MongoDB.
    """
    # ---- Metric 1: Average Watch Time ----
    avg_watch_time = compute_avg_watch_time()

    # ---- Metric 2: Watch Time Conversion Rate ----
    conversion_rate = compute_watch_time_conversion_rate()

    # ---- Prepare telemetry record ----
    telemetry_record = {
        "timestamp": datetime.utcnow(),
        "average_watch_time_sec": avg_watch_time,
        "conversion_rate_percent": conversion_rate * 100,
        "note": "Online telemetry snapshot"
    }

    # ---- Store in MongoDB telemetry collection ----
    telemetry_collection = log_db["online_evaluation_telemetry"]
    telemetry_collection.insert_one(telemetry_record)

    print("✅ Telemetry logged:", telemetry_record)
    return telemetry_record


In [12]:
log_online_telemetry()

 Average Watch Time AFTER Recommendation: 1013.50 seconds
Watch Time Conversion Rate AFTER Recommendation: 3.09%
✅ Telemetry logged: {'timestamp': datetime.datetime(2025, 3, 21, 21, 7, 26, 357414), 'average_watch_time_sec': 1013.5015527950311, 'conversion_rate_percent': 3.087934560327198, 'note': 'Online telemetry snapshot', '_id': ObjectId('67ddd50e18ddf7f3c799d9ff')}


{'timestamp': datetime.datetime(2025, 3, 21, 21, 7, 26, 357414),
 'average_watch_time_sec': 1013.5015527950311,
 'conversion_rate_percent': 3.087934560327198,
 'note': 'Online telemetry snapshot',
 '_id': ObjectId('67ddd50e18ddf7f3c799d9ff')}

In [13]:
from apscheduler.schedulers.background import BackgroundScheduler

scheduler = BackgroundScheduler()
scheduler.add_job(log_online_telemetry, 'interval', hours=1)
scheduler.start()


Run time of job "log_online_telemetry (trigger: interval[1:00:00], next run at: 2025-03-21 18:09:53 EDT)" was missed by 0:12:31.549123


 Average Watch Time AFTER Recommendation: 1313.01 seconds
Watch Time Conversion Rate AFTER Recommendation: 2.66%
✅ Telemetry logged: {'timestamp': datetime.datetime(2025, 3, 22, 2, 27, 37, 440779), 'average_watch_time_sec': 1313.0142535633909, 'conversion_rate_percent': 2.6574136763710223, 'note': 'Online telemetry snapshot', '_id': ObjectId('67de201918ddf7f3c799da00')}
 Average Watch Time AFTER Recommendation: 1318.44 seconds
Watch Time Conversion Rate AFTER Recommendation: 2.67%
✅ Telemetry logged: {'timestamp': datetime.datetime(2025, 3, 22, 3, 27, 59, 968069), 'average_watch_time_sec': 1318.4366576819407, 'conversion_rate_percent': 2.6743398781313474, 'note': 'Online telemetry snapshot', '_id': ObjectId('67de2e3f18ddf7f3c799da01')}
 Average Watch Time AFTER Recommendation: 1324.00 seconds
Watch Time Conversion Rate AFTER Recommendation: 2.63%
✅ Telemetry logged: {'timestamp': datetime.datetime(2025, 3, 22, 4, 28, 20, 969748), 'average_watch_time_sec': 1323.9994021820355, 'conversio

Run time of job "log_online_telemetry (trigger: interval[1:00:00], next run at: 2025-03-22 02:09:53 EDT)" was missed by 0:11:40.273742
Run time of job "log_online_telemetry (trigger: interval[1:00:00], next run at: 2025-03-22 03:09:53 EDT)" was missed by 0:16:02.446435


 Average Watch Time AFTER Recommendation: 1341.05 seconds
Watch Time Conversion Rate AFTER Recommendation: 2.40%
✅ Telemetry logged: {'timestamp': datetime.datetime(2025, 3, 22, 7, 30, 7, 732963), 'average_watch_time_sec': 1341.0548209775666, 'conversion_rate_percent': 2.4012158054711246, 'note': 'Online telemetry snapshot', '_id': ObjectId('67de66ff18ddf7f3c799da03')}
 Average Watch Time AFTER Recommendation: 1347.25 seconds
Watch Time Conversion Rate AFTER Recommendation: 2.34%
✅ Telemetry logged: {'timestamp': datetime.datetime(2025, 3, 22, 8, 31, 1, 400851), 'average_watch_time_sec': 1347.2468495181615, 'conversion_rate_percent': 2.3355506282335554, 'note': 'Online telemetry snapshot', '_id': ObjectId('67de754518ddf7f3c799da04')}
 Average Watch Time AFTER Recommendation: 1354.51 seconds
Watch Time Conversion Rate AFTER Recommendation: 2.27%
✅ Telemetry logged: {'timestamp': datetime.datetime(2025, 3, 22, 9, 31, 41, 817844), 'average_watch_time_sec': 1354.513981358189, 'conversion_r

Run time of job "log_online_telemetry (trigger: interval[1:00:00], next run at: 2025-03-22 09:09:53 EDT)" was missed by 0:20:23.111686


 Average Watch Time AFTER Recommendation: 1377.89 seconds
Watch Time Conversion Rate AFTER Recommendation: 2.05%
✅ Telemetry logged: {'timestamp': datetime.datetime(2025, 3, 22, 13, 30, 48, 272301), 'average_watch_time_sec': 1377.8854625550662, 'conversion_rate_percent': 2.0463670509001424, 'note': 'Online telemetry snapshot', '_id': ObjectId('67debb8818ddf7f3c799da07')}
 Average Watch Time AFTER Recommendation: 1384.17 seconds
Watch Time Conversion Rate AFTER Recommendation: 2.00%
✅ Telemetry logged: {'timestamp': datetime.datetime(2025, 3, 22, 14, 31, 10, 805876), 'average_watch_time_sec': 1384.1682544335335, 'conversion_rate_percent': 1.9977241117713997, 'note': 'Online telemetry snapshot', '_id': ObjectId('67dec9ae18ddf7f3c799da08')}
 Average Watch Time AFTER Recommendation: 1388.78 seconds
Watch Time Conversion Rate AFTER Recommendation: 1.95%
✅ Telemetry logged: {'timestamp': datetime.datetime(2025, 3, 22, 15, 30, 59, 139132), 'average_watch_time_sec': 1388.7783467446964, 'conver

Run time of job "log_online_telemetry (trigger: interval[1:00:00], next run at: 2025-03-22 14:09:53 EDT)" was missed by 0:59:27.236497


 Average Watch Time AFTER Recommendation: 1408.78 seconds
Watch Time Conversion Rate AFTER Recommendation: 1.82%
✅ Telemetry logged: {'timestamp': datetime.datetime(2025, 3, 22, 18, 33, 27, 999383), 'average_watch_time_sec': 1408.7821911828896, 'conversion_rate_percent': 1.821745647411507, 'note': 'Online telemetry snapshot', '_id': ObjectId('67df027818ddf7f3c799da0a')}
 Average Watch Time AFTER Recommendation: 1414.88 seconds
Watch Time Conversion Rate AFTER Recommendation: 1.78%
✅ Telemetry logged: {'timestamp': datetime.datetime(2025, 3, 22, 19, 34, 5, 632093), 'average_watch_time_sec': 1414.8845315904139, 'conversion_rate_percent': 1.782490974729242, 'note': 'Online telemetry snapshot', '_id': ObjectId('67df10ad18ddf7f3c799da0b')}
 Average Watch Time AFTER Recommendation: 1420.75 seconds
Watch Time Conversion Rate AFTER Recommendation: 1.74%
✅ Telemetry logged: {'timestamp': datetime.datetime(2025, 3, 22, 20, 34, 21, 879923), 'average_watch_time_sec': 1420.7453596287703, 'conversio

Run time of job "log_online_telemetry (trigger: interval[1:00:00], next run at: 2025-03-22 18:09:53 EDT)" was missed by 0:15:22.014554
